<a href="https://colab.research.google.com/github/viksingh/pythonfinance/blob/master/bollingerbandsstrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import time
import requests
from io import StringIO

start_time = time.time()

from google.colab import drive
drive.mount('/content/drive')
import sys
!{sys.executable} -m pip install PyDrive
!{sys.executable} -m pip install yfinance


from pydrive.auth import  GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import yfinance as yf

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Check if index is going up
MONEYFOREACHSTOCK = 2500
AMOUNTSPENT = 0
AMOUNTEARNERD = 0
SELLPRICE = 0
STOPLOSSFACTORINCREASINGINDEX = 0.80
STOPLOSSFACTORREDUCINGINDEX = 0.90
NUMBEROFSTOCKS = 0
CURRENTAMOUNT = 0
TOTALSTOCKSHELD = 0
STOCKVALUE = 0

TOTALSTOCKSHELD = 0

TOTALSPENT = 0
TOTALVALUE = 0
TOTALSTOPLOSSMONEY = 0


filePathIndex = '/content/drive/My Drive/Colab Notebooks/data/index2.csv'
filePathCombined = '/content/drive/My Drive/Colab Notebooks/data/stockscombined.csv'

dfindex = pd.read_csv(filePathIndex, header=0)
dfstocks = pd.read_csv(filePathCombined, header=0)

# Create a dataframe to store currently bought stocks
# Fields : STOCK, BOUGHT DATE , BOUGHT PRICE , QTY, UPDATE DATE, LAST PRICE, RoC   , BOUGHTPRICE , UPDTPRICE
# May be add an entry for each week when this gets checked
print('READ BOTH FILES')

from datetime import date, timedelta

start_date = date(2019, 1, 18)
end_date = date(2019, 12, 27)
delta = timedelta(days=7)
dfStockUniqueList = dfstocks
# dfStockUniqueList[1].unique()
# dfStockUniqueList['STOCK'] = dfStockUniqueList.groupby('STOCK')['STOCK'].rank( ascending=False)
companiesList = dfStockUniqueList.STOCK.unique()
# print(companiesList)

dfStocksCurrentlyBought = pd.DataFrame(columns=['STOCK', 'PURCHASEDATE', 'PURCHASEPRICE', 'QTY', 'UPDATEDATE', 'UPDATEPRICE', 'MAXPRICE', 'ROC', 'ACTIVE'])

while start_date <= end_date:
    # Check if index is increasing / decreasing this week
    StartDate = start_date.strftime("%Y-%m-%d")
    queryIndexWithDate = 'DATE' + '==' + "'" + StartDate + "'"
    dfNasdaqCheckIndexIncreasing = dfindex.query(queryIndexWithDate)
    if dfNasdaqCheckIndexIncreasing.size > 0:
        isIncreasing = dfNasdaqCheckIndexIncreasing['CROSS'].iloc[-1]
        print('Index : ' + str(StartDate) + ' - ' + str(isIncreasing))

        # check if any stock needs to be sold in the current week
        # loop over stocks currently bought

        # ######## SELL STOCK LOGIC ###################################################
        # ######## SELL STOCK LOGIC ###################################################

        dfstocksActive = dfStocksCurrentlyBought.query("ACTIVE == 'X'")
        stockBoughtList = dfstocksActive.STOCK.unique()

        for stockToSellCheck in stockBoughtList:
            #                print('CHECK IF WE SHOULD SELL ')

            queryGetCurrentStockDataSell = 'STOCK' + '==' + "'" + stockToSellCheck + "'"
            dfCurrentStockSell = dfstocks.query(queryGetCurrentStockDataSell)
            queryGetCurrentPriceWithDateSell = "DATE" + "<=" + "'" + StartDate + "'"
            dfCurrentStockSell = dfCurrentStockSell.query(queryGetCurrentPriceWithDateSell)
            dfCurrentStockSell = dfCurrentStockSell.tail(2)

            if dfCurrentStockSell.shape[0] == 2:

                #                    print('Checking' + str(dfCurrentStockSell.size) + 'for stock : '+ str(stock))

                CURRENTSTOCKPRICESellTHISWEEK = dfCurrentStockSell['PRICE'].iloc[1]
                CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK = dfCurrentStockSell['UPPERBOLLINGER'].iloc[1]
                CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK = dfCurrentStockSell['LOWERBOLLINGER'].iloc[1]

                CURRENTSTOCKPRICESellLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'PRICE']
                CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'UPPERBOLLINGER']
                CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK = dfCurrentStockSell.loc[dfCurrentStockSell.index[0], 'LOWERBOLLINGER']

                if (CURRENTSTOCKPRICESellTHISWEEK < CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK and CURRENTSTOCKPRICESellLASTWEEK > CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK):
                    queryGetCurrentStockDataSell = 'STOCK' + '==' + "'" + stockToSellCheck + "'"
                    dfCurrentStockDataForQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataSell)
                    STOCKQTY = dfCurrentStockDataForQty.iloc[-1]['QTY']
                    print('Sell : @ ' + str(stockToSellCheck) + ' - ' + str(StartDate) + ' - ' + str(CURRENTSTOCKPRICESellTHISWEEK) + ' - ' + str(CURRENTSTOCKPRICESellTHISWEEK * STOCKQTY))
                    dfStocksCurrentlyBought.loc[(dfStocksCurrentlyBought.STOCK == stockToSellCheck), 'ACTIVE'] = ''
                    STOCKVALUELOSS = 0
                    STOCKVALUELOSS =  STOCKVALUELOSS + CURRENTSTOCKPRICESellTHISWEEK
                    TOTALSTOCKSHELD = TOTALSTOCKSHELD - 1
                    TOTALSTOPLOSSMONEY = TOTALSTOPLOSSMONEY + CURRENTSTOCKPRICESellTHISWEEK * STOCKQTY
                    TOTALVALUE = TOTALVALUE + TOTALSTOPLOSSMONEY



                # Get current and previous week. Identify stocks

        # SELLING RULES
        # If increasing index this week : current price is higher than 0.8 MAX PRICE which will be the last row of
        # If currennt price

        # **************** IDENTIFY STOCKS TO BUY ***********************************
        # **************** IDENTIFY STOCKS TO BUY ***********************************

        if (isIncreasing == 1):
            for stockToBuy in companiesList:
                #                print("Analysing stock :  " + stock)

                queryGetCurrentStockDataBuy = 'STOCK' + '==' + "'" + stockToBuy + "'"
                dfCurrentStockBuy = dfstocks.query(queryGetCurrentStockDataBuy)
                queryGetCurrentPriceWithDateBuy = "DATE" + "<=" + "'" + StartDate + "'"
                dfCurrentStockBuy = dfCurrentStockBuy.query(queryGetCurrentPriceWithDateBuy)
                dfCurrentStockBuy = dfCurrentStockBuy.tail(2)


                if dfCurrentStockBuy.shape[0] == 2:

                    #                    print('Checking' + str(dfCurrentStockBuy.size) + 'for stock : '+ str(stock))

                    CURRENTSTOCKPRICEBUYTHISWEEK = dfCurrentStockBuy['PRICE'].iloc[1]
                    CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK = dfCurrentStockBuy['UPPERBOLLINGER'].iloc[1]
                    CURRENTSTOCKPRICELOWERBOLLINGERTHISWEEK = dfCurrentStockBuy['LOWERBOLLINGER'].iloc[1]

                    CURRENTSTOCKPRICEBUYLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'PRICE']
                    CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'UPPERBOLLINGER']
                    CURRENTSTOCKPRICELOWERBOLLINGERLASTWEEK = dfCurrentStockBuy.loc[dfCurrentStockBuy.index[0], 'LOWERBOLLINGER']

                    if (CURRENTSTOCKPRICEBUYTHISWEEK > CURRENTSTOCKPRICEUPPERBOLLINGERTHISWEEK and CURRENTSTOCKPRICEBUYLASTWEEK < CURRENTSTOCKPRICEUPPERBOLLINGERLASTWEEK and CURRENTSTOCKPRICEBUYTHISWEEK > 5):
                        NUMBEROFSTOCKS = int(MONEYFOREACHSTOCK / CURRENTSTOCKPRICEBUYTHISWEEK)

                        AMOUNTSPENTFORTHISSTOCK = NUMBEROFSTOCKS * CURRENTSTOCKPRICEBUYTHISWEEK
                        TOTALSTOCKSHELD = TOTALSTOCKSHELD + 1
                        TOTALSPENT = TOTALSPENT + AMOUNTSPENTFORTHISSTOCK
                        try:
                            CURRPRICEYAHOOFIN = str(yf.Ticker(stockToBuy).info['regularMarketPrice'])
                            CURRVALUEYAHOOFIN = str( yf.Ticker(stockToBuy).info['regularMarketPrice'] * NUMBEROFSTOCKS)
                        except Exception as e:
                            print('No current value found for ' + stockToBuy)

                        print( str(TOTALSTOCKSHELD) +' Buy : ' + str(stockToBuy) + ' - ' + str(StartDate) + ' @ ' + str(CURRENTSTOCKPRICEBUYTHISWEEK) + 'Current Price: ' + CURRPRICEYAHOOFIN + ' - ' + str(AMOUNTSPENTFORTHISSTOCK) + 'Curr Value: ' + CURRVALUEYAHOOFIN )
                        dfStocksCurrentlyBought = dfStocksCurrentlyBought.append(
                            {'STOCK': stockToBuy, 'PURCHASEDATE': StartDate, 'PURCHASEPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'QTY': NUMBEROFSTOCKS, 'UPDATEDATE': StartDate,
                             'UPDATEPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'MAXPRICE': CURRENTSTOCKPRICEBUYTHISWEEK, 'ROC': 100, 'ACTIVE': 'X'}, ignore_index=True)


    else:
        print('NO RECOMMENDATIONS AS INDEX IS GOING DOWN : ' + str(start_date))


## PRINT CURRENT VALUE
## PRINT CURRENT VALUE
    EndDate = end_date.strftime("%Y-%m-%d")




    start_date += delta

stockCurrtList = dfStocksCurrentlyBought.STOCK.unique()
for stockCurrValue in stockCurrtList:
    queryGetCurrentStockDataCurrValue = 'STOCK' + '==' + "'" + stockCurrValue + "'"
    dfCurrentStockCurrValue = dfstocks.query(queryGetCurrentStockDataCurrValue)

    queryGetCurrentPriceWithDateCurrValue = "DATE" + "==" + "'" + EndDate + "'"
    dfCurrentStockCurrValue = dfCurrentStockCurrValue.query(queryGetCurrentPriceWithDateCurrValue)
    dfCurrentStockCurrValue = dfCurrentStockCurrValue.tail(1)


# SUMMARY
    if dfCurrentStockCurrValue.shape[0] > 0:
#                PRICEFORSUMMARY = float(dfCurrentStockCurrValue['PRICE'].iloc[0])
        try:
            PRICEFORSUMMARY = float(yf.Ticker(stockCurrValue).info['regularMarketPrice'])
        except Exception as e:
            print('No current value found for ' + stockCurrValue)
        dfCurrentStockQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataCurrValue)['QTY'].iloc[0]
        STOCKVALUE = float(PRICEFORSUMMARY * dfCurrentStockQty)
        print('Value : ' + stockCurrValue + ' - ' + str(PRICEFORSUMMARY) + '- ' + str(dfCurrentStockQty) + ' - ' + str(STOCKVALUE))
        TOTALVALUE = TOTALVALUE + STOCKVALUE

print('AMOUNT SPENT : ' + str(TOTALSPENT) + ' CURR VALUE: ' + str(TOTALVALUE) + ' ROI % = ' + str( (TOTALVALUE - TOTALSPENT)/ TOTALVALUE * 100 ) + '%' )



#stockCurrtList = dfStocksCurrentlyBought.STOCK.unique()
#for stockPrintValue in stockCurrtList:
#    stock = yf.Ticker(stockPrintValue)
#    print(stockPrintValue +' : '+ str(stock.info['regularMarketPrice']))

print("--- %s seconds ---" % (time.time() - start_time))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
READ BOTH FILES
Index : 2019-01-18 - 1
1 Buy : brid - 2019-01-18 @ 23.55Current Price: 23.8 - 2496.3Curr Value: 2522.8
2 Buy : erie - 2019-01-18 @ 141.03Current Price: 167.55 - 2397.51Curr Value: 2848.3500000000004
3 Buy : ehth - 2019-01-18 @ 46.25Current Price: 99.11 - 2497.5Curr Value: 5351.94
Index : 2019-01-25 - 1
4 Buy : xlnx - 2019-01-25 @ 108.85Current Price: 98.22 - 2394.7Curr Value: 2160.84
